# Programming for Data Analysis - Project 2

**Ciaran Moran**

***

**Standard imports**

In [29]:
import matplotlib.pyplot as plt 
import random
import datetime
import pandas as pd 
import seaborn as sns
import numpy as np

## Open the .csv files
#### We can skip the first X rows in the csv as they are not pure data columns
#### Initially received the error "UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 33: invalid start byte"
###
### Looking online I tried various suggestions from 
### https://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s
###
### The working solution appears to be encoding='unicode_escape'
###
### The next issue was rows with all Nan values, which may cause issues later on.
### For this I tried keep_default_na=False and also skip_blank_lines=True from 
### https://stackoverflow.com/questions/39297878/how-to-skip-an-unknown-number-of-empty-lines-before-header-on-pandas-read-csv


In [30]:
print('df....')
df = pd.read_csv('data/41586_2008_BFnature06949_MOESM31_ESM.csv', skiprows=6, encoding='unicode_escape',  skip_blank_lines=True, keep_default_na=False)
df.describe()

print('df2....')
df2 = pd.read_csv('data/grl52461-sup-0003-supplementary.csv', skiprows=6, encoding='unicode_escape',  skip_blank_lines=True, keep_default_na=False)
df2.describe()

df....
df2....


,Depth (m),"Gasage (EDC3, yr BP)","Gasage (AICC2012, yr BP)",CO2 (ppmv),sigma mean CO2 (ppmv),Depth (m).1,"Gasage (EDC3, yr BP).1","Gasage (AICC2012, yr BP).1",CO2 (ppmv).1,Depth (m).2,...,CO2 (ppmv).19,sigma mean CO2 (ppmv).16,Depth (m).19,"Gasage (AICC2012, yr BP).13",corrected CO2 (ppmv),analytical sigma mean CO2 (ppmv),Correcting Factor (ppmv),lower bound (2 sigma) of correction F. (ppmv),upper bound (2 sigma) of correction F. (ppmv),Unnamed: 108
count,394,394,394,394,394,394,394,394,394,394,...,394,394,394,394,394,394,394,394,394,394
unique,184,184,184,149,20,373,373,349,310,74,...,314,163,393,393,383,182,306,249,385,1
top,,,,,,,,,,,...,,,,,,1.00,0.00,0.00,,
freq,211,211,211,211,211,22,22,46,22,321,...,71,71,2,2,2,7,82,143,2,394


***

## End